In [78]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, register, execute, get_backend
from qiskit.tools.visualization import circuit_drawer, plot_histogram, plot_state
from qiskit.tools.qi.qi import state_fidelity
import random
import pickle
import time
import datetime
import xlsxwriter
import circuit_builder
import importlib
importlib.reload(circuit_builder)


#setup
import sys, getpass
try:
    sys.path.append("../../") # go to parent dir
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
    print()
except:
    APItoken = getpass.getpass('Please input your token and hit enter: ')
    qx_config = {
        "APItoken": APItoken,
        "url":"https://quantumexperience.ng.bluemix.net/api"}
    print('Qconfig.py not found in qiskit-tutorial directory; Qconfig loaded using user input.')
    
#matlab stuff
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy import matrix
from math import pi

backend = "ibmqx5"


def permute(gateOrder,perm):
    permCopy = perm.copy()
    for i in range(len(perm)):
        perm[i+1] = permCopy[int(gateOrder[i])]
        
def setInput(qc,compiler):
    if compiler == 0:
        return
    elif compiler == 1:
        qc.x(qr[0])
    elif compiler == 2:
        qc.x(qr[1])
    elif compiler == 3:
        qc.x(qr)
            

# fill this in later  
# def writeToExcel():
    
    
shots = 1024

# all circuits we want to test
circuits = [["permute 2 1", "aerf", "x 1"]]
reps = 9

# fill these in to get a calculation of success rate
initialStates = [0,1,2,3]
desiredOutputs = [["01"], ["00"], ["11"], ["10"]]
validOutputs = ["00", "01", "10", "11"]

for compiler in initialStates:
    
    # main benchmarking procedure
    for circuit in circuits:
        print("Circuit is: "+str(circuit))
        print("Compiler is: "+str(compiler))
            
        avgSuccess = 0

        # maps the input info into actual physical qubits
        perm = {1: 1, 2:0}

        for trial in range(reps):
            n = 2
            qr = QuantumRegister(n)
            cr = ClassicalRegister(n)
            qc = QuantumCircuit(qr,cr)
            
            # sets our starting state to 00, 10, 01, or 11
            setInput(qc, compiler)

            # check if there's a permutation
            first = circuit[0].split()
            if first[0] == "permute":
                permute(first[1:], perm)
                circuit = circuit[1:]
            
            circuit_builder.buildCirc(qc, qr, circuit, perm)

            qc.measure(qr,cr)
            
            '''
            for i in range(n):
                qc.measure(qr[perm[i+1]], cr[i])'''


            job = execute(qc, backend,shots=shots)
            result = job.result()
            data = result.get_counts(qc)
            
            # comment this out if don't care about the individual data
            # print(data)
            
            successes = 0
            totalValid = 0
            for key, value in data.items():
                if key in desiredOutputs[compiler]:
                    successes += value
                if key in validOutputs:
                    totalValid += value

            if totalValid == 0:
                print("oops")
            else:
                # comment out if running more trials
                # print("Success rate is: "+ str(successes/totalValid))
                avgSuccess += successes/totalValid
        avgSuccess /= reps
        print("Average of success rates is: "+ str(avgSuccess)+ "\n")


Qconfig loaded from C:\Users\ddrsq\OneDrive\Quantum\qecc_benchmark\Qconfig.py.

Circuit is: ['permute 2 1', 'aerf', 'x 1']
Compiler is: 0
Some nonsense was ignored


LookupError: backend "ibmqx5" not found.